In [2]:
import numpy as np
from hmmlearn import hmm

In [3]:
np.random.seed(42)

# model = hmm.GaussianHMM(n_components=3, covariance_type="full")
# model.startprob_ = np.array([0.6, 0.3, 0.1])
# model.transmat_ = np.array([[0.7, 0.2, 0.1],
#                             [0.3, 0.5, 0.2],
#                             [0.3, 0.3, 0.4]])
# model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
# model.covars_ = np.tile(np.identity(2), (3, 1, 1))
# X, Z = model.sample(100)

# lr = hmm.GaussianHMM(n_components=3, covariance_type="diag",
#                      init_params="cm", params="cmt")
# lr.startprob_ = np.array([1.0, 0.0, 0.0])
# lr.transmat_ = np.array([[0.5, 0.5, 0.0],
#                          [0.0, 0.5, 0.5],
#                          [0.0, 0.0, 1.0]])

X1 = [[0.5], [1.0], [-1.0], [0.42], [0.24]]
X2 = [[2.4], [4.2], [0.5], [-0.24]]

X = np.concatenate([X1, X2])
print(X)
lengths = [len(X1), len(X2)]
lr = hmm.GaussianHMM(n_components=3).fit(X, lengths)
a =  lr.predict(X, lengths)
print(a)

Fitting a model with 14 free scalar parameters with only 9 data points will result in a degenerate solution.
[[ 0.5 ]
 [ 1.  ]
 [-1.  ]
 [ 0.42]
 [ 0.24]
 [ 2.4 ]
 [ 4.2 ]
 [ 0.5 ]
 [-0.24]]
[2 0 0 2 0 1 1 2 0]
